In [1]:
# SETUP ===================================
import jupyter
import warnings

from src.utils.system import boot, Notify

boot()
warnings.filterwarnings("ignore")



# PACKAGES ================================
import os
import torch
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import gymnasium as gym
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.preprocessing import  RobustScaler
from IPython.display import display

# FRAMEWORK STUFF =========================
from src.defaults import TOP2_STOCK_BY_SECTOR, FEATURE_COLS,EPISODE_LENGTH
from src.data.feature_pipeline import load_base_dataframe
from src.experiments.experiment_tracker import ExperimentTracker
from src.env.base_timeseries_trading_env import BaseSequenceAwareTradingEnv,SequenceAwareAlphaTradingEnv,SequenceAwareBaselineTradingAgent,SequenceAwareCalmarTradingEnv,SequenceAwareCumulativeTradingEnv,SequenceAwareDrawdownTradingEnv,SequenceAwareHybridTradingEnv,SequenceAwareHybridTradingEnv,SequenceAwareSharpeTradingEnv,SequenceAwareSortinoTradingEnv

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy



In [12]:
# SETUP ===================================
import jupyter
import warnings

from src.utils.system import boot, Notify

boot()
warnings.filterwarnings("ignore")



# PACKAGES ================================
import os
import torch
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import gymnasium as gym
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.preprocessing import  RobustScaler

# FRAMEWORK STUFF =========================
from src.defaults import TOP2_STOCK_BY_SECTOR, FEATURE_COLS,EPISODE_LENGTH
from src.data.feature_pipeline import load_base_dataframe
from src.experiments.experiment_tracker import ExperimentTracker
from src.env.base_timeseries_trading_env import BaseSequenceAwareTradingEnv,SequenceAwareAlphaTradingEnv,SequenceAwareBaselineTradingAgent,SequenceAwareCalmarTradingEnv,SequenceAwareCumulativeTradingEnv,SequenceAwareDrawdownTradingEnv,SequenceAwareHybridTradingEnv,SequenceAwareHybridTradingEnv,SequenceAwareSharpeTradingEnv,SequenceAwareSortinoTradingEnv
from src.env.base_trading_env import CumulativeTradingEnv

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy

from src.env.wrappers import RegimeAugmentingWrapper

In [13]:
ENV_CLASS =CumulativeTradingEnv

In [14]:
ohlcv_df = load_base_dataframe()
df = ohlcv_df.copy()

In [15]:
import pandas as pd
from tqdm import tqdm
import numpy as np

def simulate_random_policy(make_env, stock, period, n_episodes=10):
    """
    Simulates random actions on a stock-period environment.
    
    Args:
        make_env: function that returns a Gym env for (stock, period)
        stock: stock symbol
        period: tuple/list of (start_date, end_date) or similar
        n_episodes: how many random episodes to run
    
    Returns:
        mean_reward: float
        sharpe: float
        rewards: list of episode rewards
    """
    rewards = []
    for _ in range(n_episodes):
        env = make_env(stock, period)
        obs = env.reset()
        done = False
        total_reward = 0.0
        while not done:
            action = env.action_space.sample()
            obs, reward, done, *_ = env.step(action)
            total_reward += reward
        rewards.append(total_reward)
    mean_reward = np.mean(rewards)
    std_reward = np.std(rewards)
    sharpe = mean_reward / (std_reward + 1e-8)
    return mean_reward, sharpe, rewards

def get_episodes(ohlcv_df, tickers, window_size=21, step_size=21):
    """
    Splits each ticker's history into rolling episodes.
    Args:
        ohlcv_df: DataFrame with columns ['date', 'symbol', ...]
        tickers: list of tickers
        window_size: episode length (trading days)
        step_size: stride for rolling window
    Returns:
        List of (ticker, start_date, end_date)
    """
    episodes = []
    for ticker in tickers:
        df = ohlcv_df[ohlcv_df['symbol'] == ticker].sort_values('date')
        dates = df['date'].unique()
        for i in range(0, len(dates) - window_size + 1, step_size):
            start, end = dates[i], dates[i+window_size-1]
            episodes.append((ticker, start, end))
    return episodes

def make_env(stock, start_date, end_date):
    df = ohlcv_df[(ohlcv_df['symbol'] == stock) &
                  (ohlcv_df['date'] >= start_date) &
                  (ohlcv_df['date'] <= end_date)]
    return ENV_CLASS(df, episode_length=len(df), feature_cols=FEATURE_COLS)

def evaluate_episodes(episodes, n_episodes=5):
    results = []
    for (ticker, start, end) in tqdm(episodes):
        def env_fn():
            return make_env(ticker, start, end)
        mean_reward, sharpe, rewards = simulate_random_policy(lambda *_: env_fn(), ticker, (start, end), n_episodes=n_episodes)
        results.append({
            'ticker': ticker,
            'start_date': start,
            'end_date': end,
            'mean_reward': mean_reward,
            'sharpe': sharpe,
            'reward_std': np.std(rewards)
        })
    return pd.DataFrame(results)


In [17]:
tickers = ohlcv_df['symbol'].unique().tolist()
window_size = 100  # about 1 trading month
step_size = 1

# Step 1: Get episodes
episodes = get_episodes(ohlcv_df, tickers, window_size, step_size)

# Step 2: Evaluate random policy on each episode
results_df = evaluate_episodes(episodes, n_episodes=10)

# Step 3: Sort by Sharpe (descending, easiest first)
ranked_episodes = results_df.sort_values(by='sharpe', ascending=False).reset_index(drop=True)

# Print top 10
ranked_episodes.head(10)


  0%|          | 0/379810 [00:00<?, ?it/s]


ValueError: Stock MMM does not have enough data (100) for episode of length 100

In [25]:
e= SequenceAwareAlphaTradingEnv(ohlcv_df,feature_cols=['volume','return_1d'])
e.reset()[0].shape

(10, 11)